In [21]:
# Import packages for building map
import numpy as np
import pandas as pd
import sodapy as sp
import pandas_bokeh as pbk
pbk.output_notebook()
from sodapy import Socrata
import matplotlib.pyplot as plt
import geopandas as gpd
import descartes 
import json
from bokeh.io import show
from bokeh.models import (CDSView, ColorBar, ColumnDataSource, CustomJS,
                          CustomJSFilter, GeoJSONDataSource, HoverTool,
                          LinearColorMapper, Slider)
from bokeh.layouts import column, row, widgetbox
from bokeh.palettes import brewer
from bokeh.plotting import figure

Loading BokehJS ...

In [22]:
# Load file with state geometry
contiguous_cat = gpd.read_file('divisions-administratives-v1r0-comarques-1000000-20210122.shp')
cols = contiguous_cat.select_dtypes(include=[np.object]).columns
contiguous_cat[cols] = contiguous_cat[cols].apply(lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))
# Load file with each region's population
df_pobl = pd.read_excel("PoblacioComarques.xls")
# importar el nostre dataset
client = Socrata("analisi.transparenciacatalunya.cat", None)
results = client.get("rmgc-ncpb", limit=16774)
# Convert to pandas DataFrame
results_df =  gpd.GeoDataFrame.from_records(results)
contiguous_cat.head()

/var/folders/tc/wsbp34qn42s_gdgnmlhtt4480000gn/T/ipykernel_36333/1484206589.py:3: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cols = contiguous_cat.select_dtypes(include=[np.object]).columns


,CODICOMAR,NOMCOMAR,geometry
0,01,Alt Camp,"POLYGON ((373020.449 4590188.741, 372482.744 4..."
1,02,Alt Emporda,"MULTIPOLYGON (((513760.140 4662388.220, 513758..."
2,03,Alt Penedes,"POLYGON ((386593.300 4595896.200, 388616.100 4..."
3,04,Alt Urgell,"MULTIPOLYGON (((360800.100 4649174.000, 360624..."
4,05,Alta Ribagorca,"POLYGON ((327485.124 4721105.796, 327436.728 4..."


In [23]:
# num d'accidents en funció de la comarca i ordenats alfabèticament respecte del nom de la comarca
com = pd.value_counts(results_df['nomcom'])
com = com.to_frame() # convertir a Data Frame
com = com.reset_index() # introduir un index (0, 1, 2...)
com = com.sort_values("index") # la columna de velocitat es diu index, ordenar-la
com = com.rename(columns={"index": "nomcom", "nomcom": "#accidents"}) # canviar els noms correctes
Comar = gpd.GeoDataFrame(com).set_index("nomcom")
Comar = Comar.reset_index()
Comar.head() #42

,nomcom,#accidents
0,Alt Camp,146
1,Alt Emporda,563
2,Alt Penedes,397
3,Alt Urgell,187
4,Alta Ribagorca,30


In [24]:
prov = results_df[['nomcom', 'nomdem']]
prov = prov.sort_values('nomcom') 
prov.head() #16mil

,nomcom,nomdem
214,Alt Camp,Tarragona
1054,Alt Camp,Tarragona
8587,Alt Camp,Tarragona
14204,Alt Camp,Tarragona
6112,Alt Camp,Tarragona


In [25]:
merge = prov.merge(Comar, left_on='nomcom', right_on='nomcom') # associem la comarca a la província, merge 16 mil
#filtrar Bcn
merge.set_index('nomdem', inplace=True)
merge_bcn=merge.loc[['Girona']]
#eliminar duplicats
merge_bcn = merge_bcn.drop_duplicates(subset='nomcom',keep='first')
bcn=merge_bcn.reset_index()
bcn #13


,nomdem,nomcom,#accidents
0,Girona,Alt Emporda,563
1,Girona,Baix Emporda,323
2,Girona,Cerdanya,117
3,Girona,Garrotxa,170
4,Girona,Girones,507
5,Girona,Osona,480
6,Girona,Pla de l'Estany,104
7,Girona,Ripolles,60
8,Girona,Selva,595


In [26]:
#contiguous_cat = contiguous_cat.drop({0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41})
contiguous_cat = contiguous_cat.drop({0,2,3,4,5,6,7,8,10,11,12,13,15,16,17,20,21,22,24,25,26,28,29,31,32,34,35,36,37,38,39,40,41})


In [27]:
geom = contiguous_cat[["NOMCOMAR", "geometry"]] # escollir les columnes: comarca i geometria de mcp
geom = geom.sort_values('NOMCOMAR') # ordenar alfabèticament
Geom = gpd.GeoDataFrame(geom).set_index("NOMCOMAR") # tornem a posar l'índex
Geom = Geom.reset_index()
Geom = Geom.rename(columns={"NOMCOMAR": "nomcom"}) # canviar el nom
Geom.head() #13

,nomcom,geometry
0,Alt Emporda,"MULTIPOLYGON (((513760.140 4662388.220, 513758..."
1,Baix Emporda,"MULTIPOLYGON (((498479.410 4622829.390, 498478..."
2,Cerdanya,"MULTIPOLYGON (((418200.724 4699919.287, 417775..."
3,Garrotxa,"POLYGON ((462293.700 4686958.700, 462409.651 4..."
4,Girones,"POLYGON ((494081.526 4662565.750, 494214.685 4..."


In [28]:
merge2 = prov.merge(Geom, left_on='nomcom', right_on='nomcom') # associem la comarca a la província
#filtrar Bcn
merge2.set_index('nomdem', inplace=True)
merge2_bcn=merge.loc[['Girona']]
#eliminar duplicats
merge2_bcn = merge2_bcn.drop_duplicates(subset='nomcom',keep='first')
bcn2=merge_bcn.reset_index()
bcn2.head() #13

,nomdem,nomcom,#accidents
0,Girona,Alt Emporda,563
1,Girona,Baix Emporda,323
2,Girona,Cerdanya,117
3,Girona,Garrotxa,170
4,Girona,Girones,507


In [29]:
# seleccionar columna de geometries
geometry = Geom[['geometry']]
# ordenem alfabèticament i deixem bonic el dataset de poblacions
df_pobl=df_pobl.sort_values('comarca')
df_pobl=pd.DataFrame(df_pobl).set_index('comarca')


In [30]:
df_pobl=df_pobl.reset_index()
df_pobl.head() #42

,comarca,pobl,pobl_percent
0,Alt Camp,44168,0.0059
1,Alt Emporda,136981,0.0182
2,Alt Penedes,107724,0.0143
3,Alt Urgell,20201,0.0027
4,Alta Ribagorca,3813,0.0005


In [31]:
merge_norm = prov.merge(df_pobl, left_on='nomcom', right_on='comarca') # associem la comarca a la província
#filtrar Bcn
merge_norm.set_index('nomdem', inplace=True)
merge_bcn_norm=merge_norm.loc[['Girona']]
#eliminar duplicats
merge_bcn_norm = merge_bcn_norm.drop_duplicates(subset='nomcom',keep='first')
bcn_norm = merge_bcn_norm.reset_index()
bcn_norm.head() #13

,nomdem,nomcom,comarca,pobl,pobl_percent
0,Girona,Alt Emporda,Alt Emporda,136981,0.0182
1,Girona,Baix Emporda,Baix Emporda,131305,0.0174
2,Girona,Cerdanya,Cerdanya,17695,0.0023
3,Girona,Garrotxa,Garrotxa,55579,0.0074
4,Girona,Girones,Girones,186020,0.0247


In [32]:
# seleccionar columnes pobl i pobl_percent
norm = bcn_norm[['pobl', 'pobl_percent']]
combo = pd.concat([bcn, geometry, norm], axis=1) # juntem comarques, #accidents, geometria i poblacions de bcn
combo['accpercadamilhab_lld'] = combo['#accidents']/combo['pobl']*1000 # afegim col per a #accidents per cada 1000 habitants
combo


,nomdem,nomcom,#accidents,geometry,pobl,pobl_percent,accpercadamilhab_lld
0,Girona,Alt Emporda,563,"MULTIPOLYGON (((513760.140 4662388.220, 513758...",136981,0.0182,4.110059
1,Girona,Baix Emporda,323,"MULTIPOLYGON (((498479.410 4622829.390, 498478...",131305,0.0174,2.459922
2,Girona,Cerdanya,117,"MULTIPOLYGON (((418200.724 4699919.287, 417775...",17695,0.0023,6.612037
3,Girona,Garrotxa,170,"POLYGON ((462293.700 4686958.700, 462409.651 4...",55579,0.0074,3.058709
4,Girona,Girones,507,"POLYGON ((494081.526 4662565.750, 494214.685 4...",186020,0.0247,2.725513
5,Girona,Osona,480,"MULTIPOLYGON (((418580.652 4643167.275, 418307...",156592,0.0208,3.065291
6,Girona,Pla de l'Estany,104,"POLYGON ((482350.600 4671869.500, 483369.000 4...",31810,0.0042,3.269412
7,Girona,Ripolles,60,"MULTIPOLYGON (((419785.996 4664417.149, 419923...",24874,0.0033,2.412157
8,Girona,Selva,595,"MULTIPOLYGON (((482690.024 4613155.364, 482688...",166285,0.0220,3.578194


In [33]:
#triar columnes
combo = combo[['#accidents', 'nomcom', 'geometry', 'accpercadamilhab_lld']]
combo

,#accidents,nomcom,geometry,accpercadamilhab_lld
0,563,Alt Emporda,"MULTIPOLYGON (((513760.140 4662388.220, 513758...",4.110059
1,323,Baix Emporda,"MULTIPOLYGON (((498479.410 4622829.390, 498478...",2.459922
2,117,Cerdanya,"MULTIPOLYGON (((418200.724 4699919.287, 417775...",6.612037
3,170,Garrotxa,"POLYGON ((462293.700 4686958.700, 462409.651 4...",3.058709
4,507,Girones,"POLYGON ((494081.526 4662565.750, 494214.685 4...",2.725513
5,480,Osona,"MULTIPOLYGON (((418580.652 4643167.275, 418307...",3.065291
6,104,Pla de l'Estany,"POLYGON ((482350.600 4671869.500, 483369.000 4...",3.269412
7,60,Ripolles,"MULTIPOLYGON (((419785.996 4664417.149, 419923...",2.412157
8,595,Selva,"MULTIPOLYGON (((482690.024 4613155.364, 482688...",3.578194


In [34]:
combo = gpd.GeoDataFrame(combo, geometry=combo['geometry'])
type(combo)

geopandas.geodataframe.GeoDataFrame

In [35]:
# Representacio interactiva
# Input GeoJSON source that contains features for plotting
geosource = GeoJSONDataSource(geojson = combo.to_json())

In [36]:
# Define color palettes
palette = brewer['BuGn'][8]
palette = palette[::-1] # reverse order of colors so higher values have darker colors

# Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 10)

# Define custom tick labels for color bar.
tick_labels = {'0': '0', '50': '500',
               '1000':'10000', '15000':'150000',
               '20000':'20000', '25000':'250000',
               '30000':'30000', '350000':'350000',
               '40000':'40000+'}

# Create color bar.
color_bar = ColorBar(color_mapper=color_mapper, label_standoff=8,width = 500, height = 20,
                     border_line_color=None,location = (0,0), orientation = 'horizontal',
                     major_label_overrides = tick_labels)

# Create figure object.
p = figure(title = "Nombre d'accidents per cada 1000 habitants 2010-18", plot_height = 600 ,
           plot_width = 600, toolbar_location = 'below',
           tools = "pan, wheel_zoom, box_zoom, reset")
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

# Add patch renderer to figure.
states = p.patches('xs','ys', source = geosource,
                   fill_color = {'field' :'accpercadamilhab_lld', 'transform' : color_mapper},
                   line_color = 'gray', line_width = 0.45, fill_alpha = 1)

# Create hover tool
p.add_tools(HoverTool(renderers = [states],
                      tooltips = [('Comarca','@nomcom'),('# Accidents', '@accpercadamilhab_lld')]))

# Specify layout
p.add_layout(color_bar, 'below')

show(p)